Before training and testing the Naive Bayes classifier, the compressed email datasets must be extracted.
The following commands unzip the training and testing email folders so they can be accessed by the model.

In [ ]:
!unzip train-mails.zip
!unzip test-mails.zip


Archive:  train-mails.zip
   creating: train-mails/
  inflating: __MACOSX/._train-mails  
  inflating: train-mails/6-338msg1.txt  
  inflating: __MACOSX/train-mails/._6-338msg1.txt  
  inflating: train-mails/5-1300msg3.txt  
  inflating: __MACOSX/train-mails/._5-1300msg3.txt  
  inflating: train-mails/6-250msg1.txt  
  inflating: __MACOSX/train-mails/._6-250msg1.txt  
  inflating: train-mails/6-266msg3.txt  
  inflating: __MACOSX/train-mails/._6-266msg3.txt  
  inflating: train-mails/spmsgc10.txt  
  inflating: __MACOSX/train-mails/._spmsgc10.txt  
  inflating: train-mails/5-1326msg1.txt  
  inflating: __MACOSX/train-mails/._5-1326msg1.txt  
  inflating: train-mails/spmsga114.txt  
  inflating: __MACOSX/train-mails/._spmsga114.txt  
  inflating: train-mails/spmsga100.txt  
  inflating: __MACOSX/train-mails/._spmsga100.txt  
  inflating: train-mails/6-110msg3.txt  
  inflating: __MACOSX/train-mails/._6-110msg3.txt  
  inflating: train-mails/6-126msg1.txt  
  inflating: __MACOSX/train-ma

After extraction, the folders train-mails and test-mails will be available in the working directory and used for feature extraction and model training.

### Verifying Dataset Extraction

After extracting the email datasets, the following code checks that the training and testing folders were successfully created and verifies the number of email files in each folder.

In [ ]:
import os

print("Train mails:", len(os.listdir("./train-mails")))
print("Test mails:", len(os.listdir("./test-mails")))


Train mails: 702
Test mails: 260


# **CA02: This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.**

In this assignment you will ...
1. Complete the code such a way that it works correctly with this given parts of the program.
2. Explain as clearly as possible what each part of the code is doing. Use "Markdown" texts and code commenting to explain the code

IMPORTANT NOTE:

The path of your data folders 'train-mails' and 'test-mails' must be './train-mails' and './test-mails'. This means you must have your .ipynb file and these folders in the SAME FOLDER in your laptop or Google Drive. The reason for doing this is, this way the peer reviewes and I would be able to run your code from our computers using this exact same relative path, irrespective of our folder hierarchy.



The required libraries are imported to support file handling, data processing, and machine learning operations used in this assignment. The os library is used to access and manage the email files stored in directories, while numpy is used to create and manipulate numerical feature matrices. The Counter class from the collections module is used to build a dictionary of the most frequent words found in the emails. The GaussianNB model from the scikit-learn library is used to train the Naive Bayes classifier, and accuracy_score is used to evaluate the accuracy of the model on the test dataset.

In [ ]:
import os
import numpy as np
from collections import Counter

# Import all other necessary libraries. Your code below ...
#Additional libraries from `scikit-learn` are imported to train the Naive Bayes model and to evaluate its accuracy on the test dataset.
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score



# Step 1: Creating the Dictionary of Words
This function builds a dictionary of the most frequent words found across all email documents in the dataset. It reads each email file, splits the text into individual words, and counts how often each word appears. Non alphabetic tokens and single character words are removed to reduce noise in the data. The final dictionary consists of the 3,000 most common words and defines the feature space used later by the Naive Bayes classifier.



In [ ]:
def make_Dictionary(root_dir):
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words
  dictionary = Counter(all_words)
  list_to_remove = list(dictionary)

  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item]
    elif len(item) == 1:
      del dictionary[item]
  dictionary = dictionary.most_common(3000)
  return dictionary





After the dictionary is created, it serves as a reference vocabulary for feature extraction. Each email is converted into a numerical feature vector by counting how many times each dictionary word appears in the email. This process transforms the raw text data into a structured numerical format that can be used by the Naive Bayes model for training and classification.

# Step 2: Feature Extraction and Label Assignment
This function converts each email into a numerical feature representation that can be used by the Naive Bayes classifier. It reads all email files from a specified directory and creates a feature matrix where each row represents an email and each column represents a word from the previously constructed dictionary. The value stored in each cell corresponds to the frequency of a specific word within an email. At the same time, the function assigns class labels to each email based on the filename, where emails labeled as spam are assigned a value of 1 and non-spam emails are assigned a value of 0.




In [ ]:

def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels


# Enter the "path" of your "train_mails" and "test-mails" FOLDERS in this cell ...
# for example: TRAIN_DIR = '../../train-mails'
#              TEST_DIR = '../../test-mails'


TRAIN_DIR = "./train-mails"
TEST_DIR = "./test-mails"



After feature extraction is completed, the function returns two outputs: the feature matrix and the corresponding label vector. These outputs are later used to train the Naive Bayes model on the training dataset and to evaluate its performance on the test dataset.

# Step 3: Build Dictionary and Extract Features
Here we create the dictionary consisting of the top 3,000 most frequent words from the training emails. Using this dictionary, both the training and testing emails are converted into feature matrices, where each email is represented as a numerical vector of word frequencies. At the same time, the corresponding class labels are generated, with spam emails labeled as 1 and non-spam emails labeled as 0.



In [ ]:
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)



reading and processing emails from TRAIN and TEST folders


After this step, the raw email text data has been transformed into structured numerical features, making it suitable for training and evaluating the Naive Bayes classifier.

# Step 4: Train Naive Bayes, Predict Test Labels, and Evaluate Accuracy
In this step, a Gaussian Naive Bayes model is trained using the training feature matrix and corresponding labels. The trained model is then used to predict the class labels for the test feature matrix. Finally, the model’s performance is evaluated by computing the accuracy, which compares the predicted labels with the true test labels.


In [ ]:
# In this section enter your code to TRAIN the model using Naive Bayes algorithm, then PREDICT and then evaluate PERFORMANCE (Accuracy)
# Your code below ...

print("Training Model using Gaussian Naibe Bayes algorithm .....")

model = GaussianNB()
model.fit(features_matrix, labels)

print("Training completed")
print("testing trained model to predict Test Data labels")

predicted_labels = model.predict(test_features_matrix)

print("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")

accuracy = accuracy_score(test_labels, predicted_labels)
print(accuracy)






Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616


The accuracy value above represents the proportion of test emails that were correctly classified as spam or non-spam by the trained Naive Bayes model. It is calculated by comparing the predicted labels produced by the model with the true labels of the test dataset. A higher accuracy indicates better model performance and shows that the classifier is able to effectively distinguish between spam and legitimate emails.

======================= END OF PROGRAM =========================

# Conclusion
In this assignment, a spam email classification system was developed using the Gaussian Naive Bayes algorithm. The raw email text data was processed by building a dictionary of the most frequent words and converting each email into a numerical feature representation based on word frequencies. Using these features, the Naive Bayes model was trained on a labeled training dataset and evaluated on a separate test dataset. The resulting accuracy demonstrates that the model is able to effectively distinguish between spam and non-spam emails. Overall, this approach shows that probabilistic machine learning models such as Naive Bayes can perform well on text classification tasks with relatively simple feature extraction techniques.